# Advent of Code 2022

## Day 22: Monkey Map

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

Today wasn't hard, it just took a long time.

I made a `Board` object which stores the data in a `numpy` array. I also track footsteps with another array for rendering the output (commented out since it's not needed). For wrapping, if we walk off into the void, velocity reverses and walks backwards to find the other side of the void leading back in.

For part two I had to rewrite the wrapping logic to take wrapping parmameters e.g. `board.add_wrap((2, EAST), (5, WEST))` means if leaving side `2` going `EAST` then go to side `5` going `WEST`. These were then hard-coded in after drawing on a paper cube, rather than implmenting a cube-folding algorithm. The `compute_to_position_wrap` algorithm has a lot of unit tests because I wasn't sure I could get that right without a lot of tests, there's too much room for error.

### Imports

In [ ]:
from collections import namedtuple
import numpy as np

### Point

In [ ]:
# noinspection PyTypeChecker
Point = namedtuple('Point', ['x', 'y'])
# noinspection PyTypeChecker
Velocity = namedtuple('Velocity', ['dx', 'dy', 'encoding', 'name'])

### Constants and Stuff

In [ ]:
VOID = ' '
FLOOR = '.'
WALL = '#'
TILE_TYPES = [VOID, FLOOR, WALL]

NORTH = Velocity(dx=0, dy=-1, encoding=1, name='NORTH')
SOUTH = Velocity(dx=0, dy=+1, encoding=2, name='SOUTH')
EAST = Velocity(dx=1, dy=0, encoding=3, name='EAST')
WEST = Velocity(dx=-1, dy=0, encoding=4, name='WEST')

FOOTSTEP_RENDER = [None, '^', 'v', '>', '<']
FACING_NUMBER = {
    NORTH : 3,
    SOUTH : 1,
    EAST : 0,
    WEST : 2
}

### Relative Position Wrap

Only used for Part 2

In [ ]:
def compute_to_position_wrap(size: int, from_position: Point, from_velocity: Velocity, to_velocity: Velocity) -> Point:

    assert 0 < size
    assert 0 <= from_position.x < size
    assert 0 <= from_position.y < size
    assert from_velocity in {NORTH, SOUTH, EAST, WEST}
    assert to_velocity in {NORTH, SOUTH, EAST, WEST}
    x = from_position.x
    y = from_position.y
    edge = size-1

    if from_velocity == EAST:
        assert x == edge
        var = y
        if to_velocity == EAST:
            return Point(x=0, y=var)
        if to_velocity == WEST:
            return Point(x=edge, y=edge-var)
        if to_velocity == NORTH:
            return Point(x=var, y=edge)
        if to_velocity == SOUTH:
            return Point(x=edge-var, y=0)

    if from_velocity == WEST:
        assert x == 0
        var = y
        if to_velocity == EAST:
            return Point(x=0, y=edge-var)
        if to_velocity == WEST:
            return Point(x=edge, y=var)
        if to_velocity == NORTH:
            return Point(x=edge-var, y=edge)
        if to_velocity == SOUTH:
            return Point(x=var, y=0)

    if from_velocity == NORTH:
        assert y == 0
        var = x
        if to_velocity == EAST:
            return Point(x=0, y=var)
        if to_velocity == WEST:
            return Point(x=edge, y=edge-var)
        if to_velocity == NORTH:
            return Point(x=var, y=edge)
        if to_velocity == SOUTH:
            return Point(x=edge-var, y=0)

    assert from_velocity == SOUTH
    assert y == edge
    var = x
    if to_velocity == EAST:
        return Point(x=0, y=edge-var)
    if to_velocity == WEST:
        return Point(x=edge, y=var)
    if to_velocity == NORTH:
        return Point(x=edge-var, y=edge)
    if to_velocity == SOUTH:
        return Point(x=var, y=0)

# checking all the wrapping works as expected - moving east
assert compute_to_position_wrap(3, Point(x=2, y=0), EAST, EAST) == Point(x=0, y=0)
assert compute_to_position_wrap(3, Point(x=2, y=1), EAST, EAST) == Point(x=0, y=1)
assert compute_to_position_wrap(3, Point(x=2, y=2), EAST, EAST) == Point(x=0, y=2)
assert compute_to_position_wrap(3, Point(x=2, y=0), EAST, WEST) == Point(x=2, y=2)
assert compute_to_position_wrap(3, Point(x=2, y=1), EAST, WEST) == Point(x=2, y=1)
assert compute_to_position_wrap(3, Point(x=2, y=2), EAST, WEST) == Point(x=2, y=0)
assert compute_to_position_wrap(3, Point(x=2, y=0), EAST, NORTH) == Point(x=0, y=2)
assert compute_to_position_wrap(3, Point(x=2, y=1), EAST, NORTH) == Point(x=1, y=2)
assert compute_to_position_wrap(3, Point(x=2, y=2), EAST, NORTH) == Point(x=2, y=2)
assert compute_to_position_wrap(3, Point(x=2, y=0), EAST, SOUTH) == Point(x=2, y=0)
assert compute_to_position_wrap(3, Point(x=2, y=1), EAST, SOUTH) == Point(x=1, y=0)
assert compute_to_position_wrap(3, Point(x=2, y=2), EAST, SOUTH) == Point(x=0, y=0)
assert compute_to_position_wrap(4, Point(x=3, y=0), EAST, EAST) == Point(x=0, y=0)
assert compute_to_position_wrap(4, Point(x=3, y=1), EAST, EAST) == Point(x=0, y=1)
assert compute_to_position_wrap(4, Point(x=3, y=2), EAST, EAST) == Point(x=0, y=2)
assert compute_to_position_wrap(4, Point(x=3, y=3), EAST, EAST) == Point(x=0, y=3)
assert compute_to_position_wrap(4, Point(x=3, y=0), EAST, WEST) == Point(x=3, y=3)
assert compute_to_position_wrap(4, Point(x=3, y=1), EAST, WEST) == Point(x=3, y=2)
assert compute_to_position_wrap(4, Point(x=3, y=2), EAST, WEST) == Point(x=3, y=1)
assert compute_to_position_wrap(4, Point(x=3, y=3), EAST, WEST) == Point(x=3, y=0)
assert compute_to_position_wrap(4, Point(x=3, y=0), EAST, NORTH) == Point(x=0, y=3)
assert compute_to_position_wrap(4, Point(x=3, y=1), EAST, NORTH) == Point(x=1, y=3)
assert compute_to_position_wrap(4, Point(x=3, y=2), EAST, NORTH) == Point(x=2, y=3)
assert compute_to_position_wrap(4, Point(x=3, y=3), EAST, NORTH) == Point(x=3, y=3)
assert compute_to_position_wrap(4, Point(x=3, y=0), EAST, SOUTH) == Point(x=3, y=0)
assert compute_to_position_wrap(4, Point(x=3, y=1), EAST, SOUTH) == Point(x=2, y=0)
assert compute_to_position_wrap(4, Point(x=3, y=2), EAST, SOUTH) == Point(x=1, y=0)
assert compute_to_position_wrap(4, Point(x=3, y=3), EAST, SOUTH) == Point(x=0, y=0)

# checking all the wrapping works as expected - moving west
assert compute_to_position_wrap(3, Point(x=0, y=0), WEST, EAST) == Point(x=0, y=2)
assert compute_to_position_wrap(3, Point(x=0, y=1), WEST, EAST) == Point(x=0, y=1)
assert compute_to_position_wrap(3, Point(x=0, y=2), WEST, EAST) == Point(x=0, y=0)
assert compute_to_position_wrap(3, Point(x=0, y=0), WEST, WEST) == Point(x=2, y=0)
assert compute_to_position_wrap(3, Point(x=0, y=1), WEST, WEST) == Point(x=2, y=1)
assert compute_to_position_wrap(3, Point(x=0, y=2), WEST, WEST) == Point(x=2, y=2)
assert compute_to_position_wrap(3, Point(x=0, y=0), WEST, NORTH) == Point(x=2, y=2)
assert compute_to_position_wrap(3, Point(x=0, y=1), WEST, NORTH) == Point(x=1, y=2)
assert compute_to_position_wrap(3, Point(x=0, y=2), WEST, NORTH) == Point(x=0, y=2)
assert compute_to_position_wrap(3, Point(x=0, y=0), WEST, SOUTH) == Point(x=0, y=0)
assert compute_to_position_wrap(3, Point(x=0, y=1), WEST, SOUTH) == Point(x=1, y=0)
assert compute_to_position_wrap(3, Point(x=0, y=2), WEST, SOUTH) == Point(x=2, y=0)
assert compute_to_position_wrap(4, Point(x=0, y=0), WEST, EAST) == Point(x=0, y=3)
assert compute_to_position_wrap(4, Point(x=0, y=1), WEST, EAST) == Point(x=0, y=2)
assert compute_to_position_wrap(4, Point(x=0, y=2), WEST, EAST) == Point(x=0, y=1)
assert compute_to_position_wrap(4, Point(x=0, y=3), WEST, EAST) == Point(x=0, y=0)
assert compute_to_position_wrap(4, Point(x=0, y=0), WEST, WEST) == Point(x=3, y=0)
assert compute_to_position_wrap(4, Point(x=0, y=1), WEST, WEST) == Point(x=3, y=1)
assert compute_to_position_wrap(4, Point(x=0, y=2), WEST, WEST) == Point(x=3, y=2)
assert compute_to_position_wrap(4, Point(x=0, y=3), WEST, WEST) == Point(x=3, y=3)
assert compute_to_position_wrap(4, Point(x=0, y=0), WEST, NORTH) == Point(x=3, y=3)
assert compute_to_position_wrap(4, Point(x=0, y=1), WEST, NORTH) == Point(x=2, y=3)
assert compute_to_position_wrap(4, Point(x=0, y=2), WEST, NORTH) == Point(x=1, y=3)
assert compute_to_position_wrap(4, Point(x=0, y=3), WEST, NORTH) == Point(x=0, y=3)
assert compute_to_position_wrap(4, Point(x=0, y=0), WEST, SOUTH) == Point(x=0, y=0)
assert compute_to_position_wrap(4, Point(x=0, y=1), WEST, SOUTH) == Point(x=1, y=0)
assert compute_to_position_wrap(4, Point(x=0, y=2), WEST, SOUTH) == Point(x=2, y=0)
assert compute_to_position_wrap(4, Point(x=0, y=3), WEST, SOUTH) == Point(x=3, y=0)

# checking all the wrapping works as expected - moving north
assert compute_to_position_wrap(3, Point(x=0, y=0), NORTH, EAST) == Point(x=0, y=0)
assert compute_to_position_wrap(3, Point(x=1, y=0), NORTH, EAST) == Point(x=0, y=1)
assert compute_to_position_wrap(3, Point(x=2, y=0), NORTH, EAST) == Point(x=0, y=2)
assert compute_to_position_wrap(3, Point(x=0, y=0), NORTH, WEST) == Point(x=2, y=2)
assert compute_to_position_wrap(3, Point(x=1, y=0), NORTH, WEST) == Point(x=2, y=1)
assert compute_to_position_wrap(3, Point(x=2, y=0), NORTH, WEST) == Point(x=2, y=0)
assert compute_to_position_wrap(3, Point(x=0, y=0), NORTH, NORTH) == Point(x=0, y=2)
assert compute_to_position_wrap(3, Point(x=1, y=0), NORTH, NORTH) == Point(x=1, y=2)
assert compute_to_position_wrap(3, Point(x=2, y=0), NORTH, NORTH) == Point(x=2, y=2)
assert compute_to_position_wrap(3, Point(x=0, y=0), NORTH, SOUTH) == Point(x=2, y=0)
assert compute_to_position_wrap(3, Point(x=1, y=0), NORTH, SOUTH) == Point(x=1, y=0)
assert compute_to_position_wrap(3, Point(x=2, y=0), NORTH, SOUTH) == Point(x=0, y=0)
assert compute_to_position_wrap(4, Point(x=0, y=0), NORTH, EAST) == Point(x=0, y=0)
assert compute_to_position_wrap(4, Point(x=1, y=0), NORTH, EAST) == Point(x=0, y=1)
assert compute_to_position_wrap(4, Point(x=2, y=0), NORTH, EAST) == Point(x=0, y=2)
assert compute_to_position_wrap(4, Point(x=3, y=0), NORTH, EAST) == Point(x=0, y=3)
assert compute_to_position_wrap(4, Point(x=0, y=0), NORTH, WEST) == Point(x=3, y=3)
assert compute_to_position_wrap(4, Point(x=1, y=0), NORTH, WEST) == Point(x=3, y=2)
assert compute_to_position_wrap(4, Point(x=2, y=0), NORTH, WEST) == Point(x=3, y=1)
assert compute_to_position_wrap(4, Point(x=3, y=0), NORTH, WEST) == Point(x=3, y=0)
assert compute_to_position_wrap(4, Point(x=0, y=0), NORTH, NORTH) == Point(x=0, y=3)
assert compute_to_position_wrap(4, Point(x=1, y=0), NORTH, NORTH) == Point(x=1, y=3)
assert compute_to_position_wrap(4, Point(x=2, y=0), NORTH, NORTH) == Point(x=2, y=3)
assert compute_to_position_wrap(4, Point(x=3, y=0), NORTH, NORTH) == Point(x=3, y=3)
assert compute_to_position_wrap(4, Point(x=0, y=0), NORTH, SOUTH) == Point(x=3, y=0)
assert compute_to_position_wrap(4, Point(x=1, y=0), NORTH, SOUTH) == Point(x=2, y=0)
assert compute_to_position_wrap(4, Point(x=2, y=0), NORTH, SOUTH) == Point(x=1, y=0)
assert compute_to_position_wrap(4, Point(x=3, y=0), NORTH, SOUTH) == Point(x=0, y=0)

# checking all the wrapping works as expected - moving south
assert compute_to_position_wrap(3, Point(x=0, y=2), SOUTH, EAST) == Point(x=0, y=2)
assert compute_to_position_wrap(3, Point(x=1, y=2), SOUTH, EAST) == Point(x=0, y=1)
assert compute_to_position_wrap(3, Point(x=2, y=2), SOUTH, EAST) == Point(x=0, y=0)
assert compute_to_position_wrap(3, Point(x=0, y=2), SOUTH, WEST) == Point(x=2, y=0)
assert compute_to_position_wrap(3, Point(x=1, y=2), SOUTH, WEST) == Point(x=2, y=1)
assert compute_to_position_wrap(3, Point(x=2, y=2), SOUTH, WEST) == Point(x=2, y=2)
assert compute_to_position_wrap(3, Point(x=0, y=2), SOUTH, NORTH) == Point(x=2, y=2)
assert compute_to_position_wrap(3, Point(x=1, y=2), SOUTH, NORTH) == Point(x=1, y=2)
assert compute_to_position_wrap(3, Point(x=2, y=2), SOUTH, NORTH) == Point(x=0, y=2)
assert compute_to_position_wrap(3, Point(x=0, y=2), SOUTH, SOUTH) == Point(x=0, y=0)
assert compute_to_position_wrap(3, Point(x=1, y=2), SOUTH, SOUTH) == Point(x=1, y=0)
assert compute_to_position_wrap(3, Point(x=2, y=2), SOUTH, SOUTH) == Point(x=2, y=0)
assert compute_to_position_wrap(4, Point(x=0, y=3), SOUTH, EAST) == Point(x=0, y=3)
assert compute_to_position_wrap(4, Point(x=1, y=3), SOUTH, EAST) == Point(x=0, y=2)
assert compute_to_position_wrap(4, Point(x=2, y=3), SOUTH, EAST) == Point(x=0, y=1)
assert compute_to_position_wrap(4, Point(x=3, y=3), SOUTH, EAST) == Point(x=0, y=0)
assert compute_to_position_wrap(4, Point(x=0, y=3), SOUTH, WEST) == Point(x=3, y=0)
assert compute_to_position_wrap(4, Point(x=1, y=3), SOUTH, WEST) == Point(x=3, y=1)
assert compute_to_position_wrap(4, Point(x=2, y=3), SOUTH, WEST) == Point(x=3, y=2)
assert compute_to_position_wrap(4, Point(x=3, y=3), SOUTH, WEST) == Point(x=3, y=3)
assert compute_to_position_wrap(4, Point(x=0, y=3), SOUTH, NORTH) == Point(x=3, y=3)
assert compute_to_position_wrap(4, Point(x=1, y=3), SOUTH, NORTH) == Point(x=2, y=3)
assert compute_to_position_wrap(4, Point(x=2, y=3), SOUTH, NORTH) == Point(x=1, y=3)
assert compute_to_position_wrap(4, Point(x=3, y=3), SOUTH, NORTH) == Point(x=0, y=3)
assert compute_to_position_wrap(4, Point(x=0, y=3), SOUTH, SOUTH) == Point(x=0, y=0)
assert compute_to_position_wrap(4, Point(x=1, y=3), SOUTH, SOUTH) == Point(x=1, y=0)
assert compute_to_position_wrap(4, Point(x=2, y=3), SOUTH, SOUTH) == Point(x=2, y=0)
assert compute_to_position_wrap(4, Point(x=3, y=3), SOUTH, SOUTH) == Point(x=3, y=0)

### Board

In [ ]:


def turn(direction: Velocity, rotation: str) -> Velocity:
    assert direction in (NORTH, SOUTH, EAST, WEST)
    assert rotation in ('R', 'L')
    if (direction, rotation) == (NORTH, 'R'):
        return EAST
    if (direction, rotation) == (NORTH, 'L'):
        return WEST
    if (direction, rotation) == (EAST, 'R'):
        return SOUTH
    if (direction, rotation) == (EAST, 'L'):
        return NORTH
    if (direction, rotation) == (SOUTH, 'R'):
        return WEST
    if (direction, rotation) == (SOUTH, 'L'):
        return EAST
    if (direction, rotation) == (WEST, 'R'):
        return NORTH
    if (direction, rotation) == (WEST, 'L'):
        return SOUTH
    raise AssertionError(f'{direction = }, {rotation = }')

class Board:

    def __init__(self, encoded: list[str], filename):
        self.filename = filename
        self.height = len(encoded)
        self.width = max(len(line) for line in encoded)
        self.side_size = max([self.width, self.height])
        self.cube_wrapping = False
        self.wraps: dict[tuple[Point, Velocity], tuple[Point, Velocity]] = {}
        self.top_lefts: dict[int, Point] = {}
        self.hide_sides = False

        # convert encoded map data to a numpy array
        self.arr = np.zeros((self.height, self.width), dtype=int)
        self.sides = np.zeros((self.height, self.width), dtype=int)
        self.footsteps = np.zeros((self.height, self.width), dtype=int)
        for y in range(self.height):
            for x, s in enumerate(encoded[y]):
                self.arr[(y, x)] = TILE_TYPES.index(s)

    def __getitem__(self, point: Point) -> str:
        if 0 <= point.y < self.height and 0 <= point.x < self.width:
            return TILE_TYPES[self.arr[(point.y, point.x)]]
        return VOID

    def absolute_position(self, side_num: int, relative: Point):
        top_left = self.top_lefts[side_num]
        return Point(x=top_left.x + relative.x, y=top_left.y + relative.y)

    def relative_position(self, position: Point) -> tuple[int, Point]:
        side_number = self.side_number(position)
        x = position.x
        y = position.y
        x %= self.side_size
        y %= self.side_size
        return side_number, Point(x=x, y=y)

    def start_position(self):
        p = Point(x=0, y=0)
        while self[p] != FLOOR:
            p = Point(x=p.x+1, y=p.y)
        return p

    def move_part1(self, position: Point, velocity: Velocity) -> Point:
        if self[position] != FLOOR:
            raise ValueError(f'{position} is {repr(self[position])}, expected {repr(FLOOR)}')
        p = Point(x=position.x + velocity.dx, y=position.y + velocity.dy)
        if self[p] == FLOOR:
            return p
        if self[p] == WALL:
            return position
        assert self[p] == VOID
        p = position
        assert self[p] == FLOOR
        while self[p] != VOID:
            p = Point(x=p.x - velocity.dx, y=p.y - velocity.dy)
        p = Point(x=p.x + velocity.dx, y=p.y + velocity.dy)
        assert self[p] != VOID
        if self[p] == FLOOR:
            return p
        assert self[p] == WALL
        return position

    def move_part2(self, position: Point, velocity: Velocity) -> tuple[Point, Velocity]:
        if self[position] != FLOOR:
            raise ValueError(f'{position} is {repr(self[position])}, expected {repr(FLOOR)}')
        p = Point(x=position.x + velocity.dx, y=position.y + velocity.dy)
        if self[p] == FLOOR:
            return p, velocity
        if self[p] == WALL:
            return position, velocity
        assert self[p] == VOID
        p = position
        assert self[p] == FLOOR
        side_number, relative = self.relative_position(position)
        wrap_from = (side_number, velocity)
        wrap_to_side_number, wrap_to_velocity = self.wraps[wrap_from]
        relative_to_position = compute_to_position_wrap(self.side_size, relative, velocity, wrap_to_velocity)
        absolute_to_position = self.absolute_position(wrap_to_side_number, relative_to_position)
        p = absolute_to_position
        assert self[p] != VOID
        if self[p] == FLOOR:
            return p, wrap_to_velocity
        assert self[p] == WALL
        return position, velocity # reset position and velocity

    def move(self, position: Point, velocity: Velocity) -> tuple[Point, Velocity]:
        if self.cube_wrapping:
            rv = self.move_part2(position, velocity)
            assert (type(rv) == tuple and len(rv) == 2), f'expected 2-tuple but move_part2 returned {rv}'
            return rv
        rv = self.move_part1(position, velocity)
        return rv, velocity

    def map_sides(self, side_size):

        self.side_size = side_size
        self.cube_wrapping = True

        side_numbers = {}
        next_side_number = 1

        # render the map
        for y in range(self.height):
            for x in range(self.width):
                p = Point(x=x, y=y)
                if self[p] != VOID:
                    side_x = x // self.side_size
                    side_y = y // self.side_size
                    side = (side_y, side_x)
                    if side not in side_numbers:
                        side_numbers[side] = next_side_number
                        self.top_lefts[next_side_number] = p
                        next_side_number += 1
                    side_number = side_numbers[side]
                    self.sides[(p.y, p.x)] = side_number

    def add_wrap(self, from_state, to_state):
        assert from_state not in self.wraps
        self.wraps[from_state] = to_state

    def side_number(self, point):
        if 0 <= point.y < self.height and 0 <= point.x < self.width:
            if self.sides[(point.y, point.x)] == 0:
                return None
            return self.sides[(point.y, point.x)]
        return None

    def render_sides(self):

        # render the column numbers
        rv = '    '
        for col_num in range(self.width):
            rv += f'{col_num // 100 % 10}'
        rv += '\n'
        rv += '    '
        for col_num in range(self.width):
            rv += f'{col_num // 10 % 10}'
        rv += '\n'
        rv += '    '
        for col_num in range(self.width):
            rv += f'{col_num % 10}'
        rv += '\n'

        # render the map
        for y in range(self.height):
            rv += f'{y:>003} '
            for x in range(self.width):
                p = Point(x=x, y=y)
                sn = self.side_number(p)
                if sn is not None:
                    rv += str(sn)
                else:
                    rv += ' '
            rv += '\n'

        return rv


    def __str__(self):

        # render the column numbers
        rv = '    '
        for col_num in range(self.width):
            rv += f'{col_num // 100 % 10}'
        rv += '\n'
        rv += '    '
        for col_num in range(self.width):
            rv += f'{col_num // 10 % 10}'
        rv += '\n'
        rv += '    '
        for col_num in range(self.width):
            rv += f'{col_num % 10}'
        rv += '\n'

        # render the map
        for y in range(self.height):
            rv += f'{y:>003} '
            for x in range(self.width):
                p = Point(x=x, y=y)
                step = FOOTSTEP_RENDER[self.footsteps[(y, x)]]
                if step is not None:
                    rv += step
                else:
                    p = Point(x=x, y=y)
                    sn = self.side_number(p)
                    if sn is not None:
                        if not self.hide_sides and self[p] == FLOOR:
                            rv += str(sn)
                        else:
                            rv += self[p]
                    else:

                        rv += self[p]
            rv += '\n'

        return rv

    def __repr__(self):
        return str(self)

    @staticmethod
    def read(file, filename) -> 'BoardMap':
        rv = []
        line = next(file)
        line = line.strip('\r\n')
        while line.strip():
            rv.append(line)
            line = next(file)
            line = line.strip('\r\n')
        return Board(rv, filename)

### Directions

In [ ]:
def read_instructions(file):
    rv = []
    line = next(file).strip()
    prev_digit = None
    buffer = ''
    for c in line:
        digit = c in '0123456789'
        if digit == prev_digit:
            buffer += c
        else:
            if prev_digit is None:
                pass
            elif prev_digit:
                rv.append(int(buffer))
            else:
                rv.append(buffer)
            buffer = c
        prev_digit = digit
    if buffer:
        if prev_digit:
            rv.append(int(buffer))
        else:
            rv.append(buffer)
    return rv

### Input Reading

In [ ]:
def read_data(filename):
    with open(filename) as file:
        board = Board.read(file, filename)
        instructions = read_instructions(file)
        return board, instructions

### Instruction Processor

In [ ]:
def run_instructions(board, instructions):
    position = board.start_position()
    velocity = EAST
    board.footsteps[(position.y, position.x)] = velocity.encoding
    for ins in instructions:
        if type(ins) == int:
            for i in range(ins):
                rv = board.move(position, velocity)
                assert (type(rv) == tuple and (len(rv) == 2)), f'expected 2-tuple but move returned {rv}'
                next_position, next_velocity = rv
                if next_position != position:
                    position = next_position
                    velocity = next_velocity
                    board.footsteps[(position.y, position.x)] = velocity.encoding
                else:
                    break # bumped into a wall, skip the rest of moving forward until we turn
        else:
            velocity = turn(velocity, ins)
            board.footsteps[(position.y, position.x)] = velocity.encoding
    return position, velocity

def compute_password(position, velocity):
    return 1000 * (position.y+1) + 4 * (position.x+1) + FACING_NUMBER[velocity]

### Part 1

In [ ]:
INPUT_FILE = 'data/input22.txt'

In [ ]:
def main():
    board, instructions = read_data(INPUT_FILE)
    position, velocity = run_instructions(board, instructions)
    password = compute_password(position, velocity)
    print(f'The final password is {password}')

In [ ]:
if __name__ == '__main__':
    main()

### Hard-Coded Wrapping Parameters

In [ ]:
def map_cube(board):
    if board.filename == 'data/example22.txt':

        board.map_sides(side_size=4)

        board.add_wrap((1, WEST), (3, SOUTH))
        board.add_wrap((1, NORTH), (2, SOUTH))
        board.add_wrap((1, EAST), (6, WEST))

        board.add_wrap((2, SOUTH), (5, NORTH))
        board.add_wrap((2, NORTH), (1, SOUTH))
        board.add_wrap((2, WEST), (6, NORTH))

        board.add_wrap((3, NORTH), (1, EAST))
        board.add_wrap((3, SOUTH), (5, EAST))

        board.add_wrap((4, EAST), (6, SOUTH))

        board.add_wrap((5, WEST), (3, NORTH))
        board.add_wrap((5, SOUTH), (2, NORTH))

        board.add_wrap((6, NORTH), (4, WEST))
        board.add_wrap((6, EAST), (1, EAST))
        board.add_wrap((6, SOUTH), (2, EAST))

    elif board.filename == 'data/input22.txt':

        board.map_sides(side_size=50)

        board.add_wrap((1, NORTH), (6, EAST))
        board.add_wrap((1, WEST), (4, EAST))

        board.add_wrap((2, NORTH), (6, NORTH))
        board.add_wrap((2, EAST), (5, WEST))
        board.add_wrap((2, SOUTH), (3, WEST))

        board.add_wrap((3, WEST), (4, SOUTH))
        board.add_wrap((3, EAST), (2, NORTH))

        board.add_wrap((4, NORTH), (3, EAST))
        board.add_wrap((4, WEST), (1, EAST))

        board.add_wrap((5, EAST), (2, WEST))
        board.add_wrap((5, SOUTH), (6, WEST))

        board.add_wrap((6, WEST), (1, SOUTH))
        board.add_wrap((6, SOUTH), (2, SOUTH))
        board.add_wrap((6, EAST), (5, NORTH))

    else:
        raise AssertionError('unknown data file')

### Part 2

In [ ]:
INPUT_FILE = 'data/input22.txt'

In [ ]:
def main():

    board, instructions = read_data(INPUT_FILE)

    # map the cube wrapping
    map_cube(board)

    position, velocity = run_instructions(board, instructions)
    password = compute_password(position, velocity)

    print(f'The final password is {password}')

In [ ]:
if __name__ == '__main__':
    main()